In [ ]:
# 240630
from glob import glob
import pickle
from openai  import OpenAI
import os
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor
from itertools import product
import json
from torch.utils.data import Dataset, DataLoader
from format import *

client = OpenAI(api_key="")

In [2]:
class ModelResponder:
    def __init__(self, model_path, ques_path_list, context_list, prompt_func_list=None, path=None):
        self.batch_size = 350
        self.model_path = model_path
        if path is not None:
            self.path = path
        else:
            self.path = os.path.basename(model_path)

        if not prompt_func_list:
            default_prompt_func = lambda inst,ques: f"{ques}"
            prompt_func_list = [default_prompt_func]
        self.prompt_func_list = prompt_func_list

        indexed_ques_paths = list(enumerate(ques_path_list))
        indexed_prompt_funcs = list(enumerate(self.prompt_func_list))
        indexed_contexts = list(enumerate(context_list))
        self.combinations = list(product(indexed_ques_paths, indexed_prompt_funcs, indexed_contexts))

    def process_and_update(self, ques_dict, inst, prompt_func, batch_pbar):
        max_retries = 5
        retries = 0
        ques=ques_dict['input']
        while retries < max_retries:
            try:
                response = client.chat.completions.create(
                    model=self.model_path,
                    messages=[
                        {"role": "system", "content": inst},
                        {"role": "user", "content": ques}
                    ],
                    temperature=0.0000000000000000000000000000000000000000001,
                    top_p=0.0000000000000000000000000000000000000000001,
                    seed=100,
                    )
                batch_pbar.update(1)
                output = response.choices[0].message.content
                ques_dict['response'] = output
                return ques_dict
            except Exception as e:
                current = f"{ques_dict['year']}-{ques_dict['session']}-{ques_dict['question_number']}"
                retries += 1
                print(e)
                print(f"Retrying '{current}'... {retries}/{max_retries}")
                time.sleep(60)
                if retries == max_retries:
                    print(f"Failed to generate response for '{current}', skipping...")
                    ques_dict['response'] = None
                    return ques_dict
                
    def process_files(self):
        for (ques_idx, ques_path), (prompt_idx, prompt_func), (context_idx, inst) in self.combinations:
            with open(ques_path) as f:
                exam = json.load(f)
            filename=f'output/{self.path} [f{prompt_idx+1}_p{context_idx+1}_q{ques_idx+1}].json'
            dataset = ExamDataset(exam, inst, prompt_func)
            dataloader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False, num_workers=0, collate_fn=collate_fn)
            with tqdm(total=len(dataloader),desc=filename, leave=True, position=1) as pbar:
                for i, batch in enumerate(dataloader):
                    with ThreadPoolExecutor() as executor:
                        with tqdm(total=len(batch), desc=f"Batch {i}", leave=True, position=0) as batch_pbar:
                            results = list(executor.map(lambda ques: self.process_and_update(ques, inst, prompt_func, batch_pbar), batch))
                    if os.path.exists(filename):
                        with open(filename, 'r') as file:
                            resp = json.load(file)
                    else:
                        resp = []
                        os.makedirs(os.path.dirname(filename), exist_ok=True)
                    for result in results:
                        resp.append(result)
                    with open(filename, 'w', encoding='utf-8') as f:
                        json.dump(resp, f, indent=4, ensure_ascii=False)
                    pbar.update(1)

In [3]:
model_path="gpt-4o-2024-05-13"

run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-4o-2024-05-13 [f1_p1_q2].json: 100%|██████████| 6/6 [01:06<00:00, 11.04s/it]


In [4]:
model_path="gpt-4-turbo-2024-04-09"

run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-4-turbo-2024-04-09 [f1_p1_q2].json: 100%|██████████| 6/6 [01:23<00:00, 13.91s/it]


In [5]:
model_path="gpt-3.5-turbo-0125"

run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-3.5-turbo-0125 [f1_p1_q2].json: 100%|██████████| 6/6 [00:57<00:00,  9.66s/it]


In [6]:
model_path="gpt-4-0613"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-4-0613 [f1_p1_q2].json: 100%|██████████| 6/6 [01:30<00:00, 15.08s/it]


In [7]:
model_path="gpt-3.5-turbo-0613"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-3.5-turbo-0613 [f1_p1_q2].json: 100%|██████████| 6/6 [01:14<00:00, 12.43s/it]


In [8]:
# model_path="gpt-4-0314"
# run = ModelResponder(model_path, ques_path_list, inst_list)
# run.process_files()

In [9]:
model_path="gpt-3.5-turbo-0301"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-3.5-turbo-0301 [f1_p1_q2].json: 100%|██████████| 6/6 [00:52<00:00,  8.80s/it]


In [3]:
model_path="gpt-4-1106-preview"
run = ModelResponder(model_path, exam_list, inst_list)
run.process_files()

output/gpt-4-1106-preview [f1_p1_q2].json: 100%|██████████| 6/6 [01:51<00:00, 18.66s/it]
